In [1]:
import cv2
import math
import time
import json
import numpy as np
from dataclasses import dataclass, field
from typing import Dict, List, Optional, Tuple

# ===== Ring size lookup (US) =====
RING_SIZE_TABLE = [
    {"us": 3.0,  "diam": 14.07, "circ": 44.20},
    {"us": 3.5,  "diam": 14.48, "circ": 45.51},
    {"us": 4.0,  "diam": 14.86, "circ": 46.82},
    {"us": 4.5,  "diam": 15.27, "circ": 48.13},
    {"us": 5.0,  "diam": 15.70, "circ": 49.45},
    {"us": 5.5,  "diam": 16.10, "circ": 50.76},
    {"us": 6.0,  "diam": 16.51, "circ": 52.07},
    {"us": 6.5,  "diam": 16.92, "circ": 53.38},
    {"us": 7.0,  "diam": 17.32, "circ": 54.69},
    {"us": 7.5,  "diam": 17.73, "circ": 56.01},
    {"us": 8.0,  "diam": 18.14, "circ": 57.32},
    {"us": 8.5,  "diam": 18.54, "circ": 58.63},
    {"us": 9.0,  "diam": 18.95, "circ": 59.94},
    {"us": 9.5,  "diam": 19.35, "circ": 61.26},
    {"us": 10.0, "diam": 19.76, "circ": 62.57},
    {"us": 10.5, "diam": 20.17, "circ": 63.88},
    {"us": 11.0, "diam": 20.57, "circ": 65.19},
    {"us": 11.5, "diam": 20.98, "circ": 66.50},
    {"us": 12.0, "diam": 21.39, "circ": 67.82},
    {"us": 12.5, "diam": 21.79, "circ": 69.13},
    {"us": 13.0, "diam": 22.20, "circ": 70.44},
    {"us": 13.5, "diam": 22.61, "circ": 71.75},
    {"us": 14.0, "diam": 23.01, "circ": 73.07},
]

def interp_ring_size_from_circ(c_mm: float) -> Dict[str, float]:
    arr = sorted(RING_SIZE_TABLE, key=lambda s: s["circ"])
    if c_mm <= arr[0]["circ"]:
        us = arr[0]["us"]
    elif c_mm >= arr[-1]["circ"]:
        us = arr[-1]["us"]
    else:
        for i in range(len(arr) - 1):
            a, b = arr[i], arr[i+1]
            if a["circ"] <= c_mm <= b["circ"]:
                t = (c_mm - a["circ"]) / (b["circ"] - a["circ"] + 1e-9)
                us = a["us"] + t * (b["us"] - a["us"])
                break
    return {"us": round(us, 2), "eu": round(c_mm, 1), "diam": round(c_mm / math.pi, 2)}

# ===== math helpers =====
def unit(v: np.ndarray) -> np.ndarray:
    n = np.linalg.norm(v);  return v if n == 0 else v / n
def rot90(v: np.ndarray) -> np.ndarray:
    return np.array([-v[1], v[0]], dtype=np.float32)
def trimmed_mean(values: List[float], trim: int = 1) -> Optional[float]:
    if not values: return None
    vs = sorted(values)
    if len(vs) > 2*trim: vs = vs[trim:-trim]
    return float(np.mean(vs)) if vs else None

# ===== robust width on a cross-section =====
def sample_line_gray(gray: np.ndarray, center: Tuple[float, float], normal: np.ndarray, half_len_px: int) -> np.ndarray:
    cx, cy = center
    coords = []
    for t in range(-half_len_px, half_len_px + 1):
        x = cx + normal[0] * t
        y = cy + normal[1] * t
        ix = max(0, min(gray.shape[1] - 1, int(round(x))))
        iy = max(0, min(gray.shape[0] - 1, int(round(y))))
        coords.append(gray[iy, ix])
    sig = np.array(coords, dtype=np.float32)
    return cv2.GaussianBlur(sig.reshape(1, -1), (1, 7), 0).flatten()

def subpixel_peak(signal: np.ndarray, idx: int) -> float:
    if idx <= 0 or idx >= len(signal)-1: return float(idx)
    y0, y1, y2 = signal[idx-1], signal[idx], signal[idx+1]
    denom = (y0 - 2*y1 + y2)
    if abs(denom) < 1e-6: return float(idx)
    delta = 0.5 * (y0 - y2) / denom
    return float(idx + delta)

def measure_width_px(gray: np.ndarray, center: Tuple[float, float], normal: np.ndarray,
                     half_len_px: int = 90, grad_thresh: float = 6.0) -> Tuple[Optional[float], float]:
    line = sample_line_gray(gray, center, normal, half_len_px)
    grad = np.abs(np.diff(line))
    mid = len(line) // 2
    left_idx = np.argmax(grad[:max(1, mid - 1)])
    right_idx = np.argmax(grad[mid:]) + mid
    left_val = grad[left_idx] if left_idx < len(grad) else 0.0
    right_val = grad[right_idx] if right_idx < len(grad) else 0.0
    if left_val < grad_thresh or right_val < grad_thresh or left_idx >= mid or right_idx <= mid:
        return None, 0.0
    left_sp = subpixel_peak(grad, left_idx)
    right_sp = subpixel_peak(grad, right_idx)
    width_px = (right_sp - left_sp)
    strength = min(left_val, right_val) / (max(np.max(grad), 1e-6))
    symmetry = 1.0 - abs((mid - left_sp) - (right_sp - mid)) / (half_len_px + 1e-6)
    q = float(max(0.0, min(1.0, 0.5*strength + 0.5*symmetry)))
    return float(width_px), q

# ===== MediaPipe Hands =====
try:
    import mediapipe as mp
except Exception as e:
    raise SystemExit("Failed to import mediapipe. Install: pip install mediapipe\n" + str(e))
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
mp_styles = mp.solutions.drawing_styles

FINGERS = {"Thumb":(1,2), "Index":(5,6), "Middle":(9,10), "Ring":(13,14), "Pinky":(17,18)}

# ===== ArUco detection (OpenCV contrib) =====
def get_aruco_detector():
    try:
        ar_dict = cv2.aruco.getPredefinedDictionary(cv2.aruco.DICT_4X4_50)
        params = cv2.aruco.DetectorParameters()
        detector = cv2.aruco.ArucoDetector(ar_dict, params)
        return detector, ar_dict, params
    except Exception:
        ar_dict = cv2.aruco.Dictionary_get(cv2.aruco.DICT_4X4_50)
        params = cv2.aruco.DetectorParameters_create()
        return None, ar_dict, params

def detect_aruco(gray: np.ndarray, detector, ar_dict, params):
    if detector is not None:
        corners, ids, _ = detector.detectMarkers(gray)
    else:
        corners, ids, _ = cv2.aruco.detectMarkers(gray, ar_dict, parameters=params)
    return corners, ids

@dataclass
class ScaleState:
    px_per_mm: Optional[float] = None
    side_mm: float = 50.0
    last_values: List[float] = field(default_factory=list)
    conf: float = 0.0
    def update(self, side_px_values: List[float]):
        if not side_px_values:
            self.conf = 0.0;  return
        avg_px = float(np.mean(side_px_values))
        px_per_mm = avg_px / self.side_mm
        self.last_values.append(px_per_mm)
        if len(self.last_values) > 30: self.last_values = self.last_values[-30:]
        self.px_per_mm = float(np.median(self.last_values))
        v = np.var(self.last_values[-10:]) if len(self.last_values) >= 10 else np.var(self.last_values)
        self.conf = float(max(0.0, min(1.0, 1.0 / (1.0 + 2000.0 * v))))

@dataclass
class EMA:
    alpha: float
    value: Optional[float] = None
    def update(self, x: Optional[float]) -> Optional[float]:
        if x is None: return self.value
        self.value = x if self.value is None else (self.alpha*x + (1-self.alpha)*self.value);  return self.value
    def reset(self): self.value = None

class RingSizerPro:
    def __init__(self, cam_index: int = 0, aruco_side_mm: float = 50.0, calib_npz: str = "camera_params.npz"):
        self.cap = cv2.VideoCapture(cam_index)
        if not self.cap.isOpened():
            raise SystemExit("Cannot open camera.")
        self.scale = ScaleState(side_mm=aruco_side_mm)
        self.detector, self.ar_dict, self.params = get_aruco_detector()
        self.smoothers = {n: EMA(alpha=0.3) for n in FINGERS.keys()}
        self.knuckle_smoothers = {n: EMA(alpha=0.3) for n in FINGERS.keys()}
        self.window = "Ring Sizer Pro v2"
        cv2.namedWindow(self.window)
        self.last_snapshot = {}

        # ---- Load camera intrinsics (if available) and build undistort map
        self.undistort_enabled = True
        self.K = None; self.D = None; self.newK = None; self.map1 = None; self.map2 = None
        self.rms = None
        self.calib_path = calib_npz

    def try_build_undistort_map(self, frame_shape_wh):
        # Build maps only once (or when size changes)
        if not os.path.exists(self.calib_path):
            return False
        data = np.load(self.calib_path, allow_pickle=True)
        K = data.get("camera_matrix"); D = data.get("dist_coeffs")
        self.rms = float(data.get("rms", 0.0))
        if K is None or D is None: return False
        h, w = frame_shape_wh[1], frame_shape_wh[0]
        newK, _ = cv2.getOptimalNewCameraMatrix(K, D, (w, h), alpha=0.0)  # 0=crop, 1=keep all
        map1, map2 = cv2.initUndistortRectifyMap(K, D, None, newK, (w, h), cv2.CV_16SC2)
        self.K, self.D, self.newK, self.map1, self.map2 = K, D, newK, map1, map2
        return True

    def draw_text(self, frame, text_lines, x=10, y=24, lh=22):
        for i, t in enumerate(text_lines):
            yy = y + i*lh
            cv2.putText(frame, t, (x, yy), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,0,0), 3, cv2.LINE_AA)
            cv2.putText(frame, t, (x, yy), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255,255,255), 1, cv2.LINE_AA)

    def estimate_scale_from_aruco(self, gray, frame):
        corners, ids = detect_aruco(gray, self.detector, self.ar_dict, self.params)
        side_lengths = []
        if ids is not None:
            cv2.aruco.drawDetectedMarkers(frame, corners, ids)
            for c in corners:
                c = c.reshape(-1,2)
                s = 0.0
                for k in range(4):
                    p1 = c[k]; p2 = c[(k+1)%4]
                    s += np.linalg.norm(p2 - p1)
                side_lengths.append(s / 4.0)
        self.scale.update(side_lengths)

    def measure_finger(self, gray, pts2d, mcp_idx, pip_idx, frame_vis) -> Tuple[Optional[float], float, Optional[float], float]:
        p_mcp = pts2d[mcp_idx];  p_pip = pts2d[pip_idx]
        axis = unit(p_pip - p_mcp);  normal = unit(np.array([-axis[1], axis[0]], dtype=np.float32))
        ts = [0.35, 0.45, 0.55]  # ring zone centers
        widths, quals = [], []
        for t in ts:
            c = (p_mcp + t*(p_pip - p_mcp)).astype(np.float32)
            wpx, q = measure_width_px(gray, (float(c[0]), float(c[1])), normal, half_len_px=90, grad_thresh=6.0)
            if wpx is not None:
                widths.append(wpx);  quals.append(q)
            a = (int(c[0] - normal[0]*90), int(c[1] - normal[1]*90))
            b = (int(c[0] + normal[0]*90), int(c[1] + normal[1]*90))
            cv2.line(frame_vis, a, b, (0, 255, 255), 1)

        diam_px = trimmed_mean(widths, trim=1) if len(widths) >= 2 else (widths[0] if widths else None)
        q_diam = float(np.mean(quals)) if quals else 0.0

        ts_knuckle = np.linspace(0.55, 0.85, 4)
        k_widths, k_quals = [], []
        for t in ts_knuckle:
            c = (p_mcp + t*(p_pip - p_mcp)).astype(np.float32)
            wpx, q = measure_width_px(gray, (float(c[0]), float(c[1])), normal, half_len_px=100, grad_thresh=6.0)
            if wpx is not None:
                k_widths.append(wpx);  k_quals.append(q)
        knuckle_px = max(k_widths) if k_widths else None
        q_knuckle = float(np.mean(k_quals)) if k_quals else 0.0
        return diam_px, q_diam, knuckle_px, q_knuckle

    def run(self):
        import os
        with mp_hands.Hands(static_image_mode=False, max_num_hands=1, model_complexity=1,
                            min_detection_confidence=0.6, min_tracking_confidence=0.5) as hands:
            built_maps = False
            while True:
                ok, frame = self.cap.read()
                if not ok: break
                frame = cv2.flip(frame, 1)

                # Build undistort maps (first frame or if size changes)
                if not built_maps:
                    built_maps = self.try_build_undistort_map((frame.shape[1], frame.shape[0]))

                # Undistort (if available & enabled)
                if built_maps and self.undistort_enabled and self.map1 is not None:
                    frame_u = cv2.remap(frame, self.map1, self.map2, interpolation=cv2.INTER_LINEAR)
                else:
                    frame_u = frame

                gray = cv2.cvtColor(frame_u, cv2.COLOR_BGR2GRAY)

                # 1) ArUco-based scale
                self.estimate_scale_from_aruco(gray, frame_u)

                # 2) Hand landmarks on undistorted RGB
                rgb = cv2.cvtColor(frame_u, cv2.COLOR_BGR2RGB)
                results = hands.process(rgb)

                overlay = [
                    f"Undistort: {'ON' if (built_maps and self.undistort_enabled) else 'OFF'}"
                    + (f" (RMS={self.rms:.3f})" if self.rms is not None else ""),
                    f"Calibration: {'OK' if self.scale.px_per_mm else '…'} "
                    + (f"px/mm={self.scale.px_per_mm:.3f}" if self.scale.px_per_mm else "show ArUco (50.0 mm)"),
                    f"Calib confidence: {self.scale.conf:.2f}",
                    "Keys: Q quit | S save JSON | R reset smoothing | U toggle undistort"
                ]

                if results.multi_hand_landmarks:
                    for hand in results.multi_hand_landmarks:
                        mp_drawing.draw_landmarks(
                            frame_u, hand, mp_hands.HAND_CONNECTIONS,
                            mp_styles.get_default_hand_landmarks_style(),
                            mp_styles.get_default_hand_connections_style()
                        )
                        h, w = gray.shape
                        pts = [np.array([lm.x * w, lm.y * h], dtype=np.float32) for lm in hand.landmark]
                        per_finger = {}
                        for name, (mcp_idx, pip_idx) in FINGERS.items():
                            diam_px, q_d, kn_px, q_k = self.measure_finger(gray, pts, mcp_idx, pip_idx, frame_u)
                            d_px_s = self.smoothers[name].update(diam_px)
                            k_px_s = self.knuckle_smoothers[name].update(kn_px)
                            if self.scale.px_per_mm and d_px_s:
                                diam_mm = d_px_s / self.scale.px_per_mm
                                circ_mm = diam_mm * math.pi
                                sz = interp_ring_size_from_circ(circ_mm)
                                advisory = None
                                if k_px_s and k_px_s > d_px_s * 1.07:
                                    advisory = "+0.5 US (knuckle larger)"
                                per_finger[name] = {
                                    "diameter_mm": round(diam_mm, 2),
                                    "circumference_mm": round(circ_mm, 2),
                                    "us_size": sz["us"],
                                    "eu_size": sz["eu"],
                                    "quality": round(0.5*q_d + 0.5*self.scale.conf, 2),
                                    "knuckle_hint": advisory
                                }
                            else:
                                per_finger[name] = {"status": "calibrating" if not self.scale.px_per_mm else "measuring"}

                        overlay.append("— Finger sizes —")
                        for fname in ["Thumb","Index","Middle","Ring","Pinky"]:
                            info = per_finger.get(fname, {})
                            if "diameter_mm" in info:
                                line = (f"{fname}: Ø {info['diameter_mm']} mm | C {info['circumference_mm']} mm | "
                                        f"US {info['us_size']} | EU {info['eu_size']} | q={info['quality']}")
                                if info.get("knuckle_hint"): line += f" | {info['knuckle_hint']}"
                                overlay.append(line)
                            else:
                                overlay.append(f"{fname}: {info.get('status','—')}")
                        self.last_snapshot = {
                            "timestamp": int(time.time()),
                            "px_per_mm": self.scale.px_per_mm,
                            "calib_conf": self.scale.conf,
                            "fingers": per_finger
                        }
                else:
                    overlay.append("No hand detected. Show one hand, palm down, fingers apart.")

                # Draw overlay
                h, w, _ = frame_u.shape
                cv2.rectangle(frame_u, (10, h-60), (280, h-15), (0,0,0), -1)
                cv2.putText(frame_u, f"Calib q={self.scale.conf:.2f}", (18, h-25),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255,255,255), 1, cv2.LINE_AA)
                for i, t in enumerate(overlay):
                    y = 24 + i*22
                    cv2.putText(frame_u, t, (10, y), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,0,0), 3, cv2.LINE_AA)
                    cv2.putText(frame_u, t, (10, y), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255,255,255), 1, cv2.LINE_AA)

                cv2.imshow(self.window, frame_u)
                k = cv2.waitKey(1) & 0xFF
                if k == ord('q'):
                    break
                elif k == ord('s'):
                    ts = int(time.time())
                    fp = f"ring_sizes_snapshot_{ts}.json"
                    with open(fp, "w", encoding="utf-8") as f:
                        json.dump(self.last_snapshot, f, ensure_ascii=False, indent=2)
                    print(f"[Saved] {fp}")
                elif k == ord('r'):
                    for e in self.smoothers.values(): e.reset()
                    for e in self.knuckle_smoothers.values(): e.reset()
                    print("[Smoothing reset]")
                elif k == ord('u'):
                    self.undistort_enabled = not self.undistort_enabled
                    print(f"[Undistort] {'ON' if self.undistort_enabled else 'OFF'}")

        self.cap.release()
        cv2.destroyAllWindows()

if __name__ == "__main__":
    import os
    RingSizerPro(cam_index=0, aruco_side_mm=50.0, calib_npz="camera_params.npz").run()


C:\ProgramData\anaconda3\envs\handsize2\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


[Undistort] OFF
[Undistort] ON
[Undistort] OFF
[Saved] ring_sizes_snapshot_1758677813.json
[Saved] ring_sizes_snapshot_1758677832.json


In [3]:
pip install opencv-contrib-python mediapipe numpy


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
